In [ ]:
# 고객센터 챗봇
import os
import json
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()

# 가상의 고객 데이터
customer_reviews = [
    {'id' : 1,
     'product': '무선 청소기',
     'date' : '2025-10-03',
     'content' : '너무 시끄럽고 충전이 잘 안돼요. 그냥 환불하고 싶어요. 박스는 버렸어요.'
     },
    {'id' : 2,
     'product': '여름용 셔츠',
     'date' : '2025-05-13',
     'content' : '사이즈가 광고랑 달라요. 배송도 느리고. 교환가능한가요? 구매한지는 한달이 조금 넘었습니다.'
     },
    {'id' : 3,
     'product': '블루투스 키보드',
     'date' : '2025-07-03',
     'content' : '블루투스 연결이 안돼요. 교환 또는 환불해주세요. 박스 모두 가지고 있어요.'
     },          
]

# 분류 - 판단 - 대응
# 회사의 환불/보상 규정 (Knowledge Base for RAG)
policy_doc = """
[다판다 교환/환불 및 보상규정]
1. 단순 변심 환불 : 구매 후 7일 이내 가능, 단, 제품 박스가 온전해야 함
2. 제품 불량 환불 : 구매 후 30일 이내 가능, 박스유무 상관없음
3. 불만족 보장 : 구매 확정 후에는 환불 불가, 소정의 적립금(2,000원) 지급 가능
4. AS 규정 : 구매 후 1년 이내 무상 AS 가능
"""

def ask_gpt_json(prompt):     
    response = client.chat.completions.create(
        model='gpt-5-nano',
        messages= [{'role':'user', 'content': prompt}],
        response_format={'type' : 'json_object'}   # json 포맷 받고 싶으면 꼭 지정해야함.
        )
    try:
        return response.choices[0].message.content
    
    except:
        return None


In [5]:
# 단순히 요약이 아닌, 카테고리, 감정점수, 핵심요구사항을 뽑아야함
def analyze_review(text):
    prompt = f'''
    너는 쇼핑몰 리뷰 전문가야, 아래 리뷰를 분석해서 다음 json 형식으로 출력
    
    [출력형식]
    {{
        'sentiment' : '긍정/부정/중립 중 하나',
        'category' : '배송/품질/서비스/기타 중 하나',
        'request_type' : '환불/교환/보상/AS/단순후기 중',
        'key_issue' : "핵심 불만 사항(10자 이내)"
    }}


    [리뷰]
    {text}
    '''

    return json.loads(ask_gpt_json(prompt))  # 문자열 형태의 dict를 실제 객체로 변환

analyzed_data_list =[]
for item in customer_reviews:
    analysis = analyze_review(item['content'])
    # 원본 데이터에 분석 결과 합치기
    merged_data = {**item, **analysis}  #**는 파이썬에서 딕셔너리를 풀어서 합치는(unpacking) 연산자
    analyzed_data_list.append(merged_data)

analyzed_data_list

[{'id': 1,
  'product': '무선 청소기',
  'date': '2025-10-03',
  'content': '너무 시끄럽고 충전이 잘 안돼요. 그냥 환불하고 싶어요. 박스는 버렸어요.',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '환불',
  'key_issue': '충전 불량'},
 {'id': 2,
  'product': '여름용 셔츠',
  'date': '2025-05-13',
  'content': '사이즈가 광고랑 달라요. 배송도 느리고. 교환가능한가요? 구매한지는 한달이 조금 넘었습니다.',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '교환',
  'key_issue': '사이즈불일치'},
 {'id': 3,
  'product': '블루투스 키보드',
  'date': '2025-07-03',
  'content': '블루투스 연결이 안돼요. 교환 또는 환불해주세요. 박스 모두 가지고 있어요.',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '교환',
  'key_issue': '블루투스연결불가'}]

In [7]:
# AI가 멋대로 판단하지 않고, 근거를 기반으로 생각하게 만들어야 함
def make_decision(customer_data, policy_doc):
    prompt = f'''
    너는 베테랑 CS 매니저야
    아래 [고객데이터]와 [회사규정]을 비교해서 요구사항을 들어줄 수 있는지 판단해
    
    [회사규정]
    {policy_doc}
    
    [고객데이터]
    - 구매일 : {customer_data['date']}
    - 오늘날짜 : 2025-11-24 (기준)
    - 요청유형 : {customer_data['request_type']}
    - 내용 : {customer_data['content']}

    [지시사항]
    1. 규정의 몇 번 조항에 해당되는지 먼저 생각할 것 (COT)
    2. 날짜를 계산해서 기간 내인지 확인할 것
    3. 최종 결정을 json으로 출력할 것

    [출력형식]
    {{
        'reasoning' : '판단 근거(단계별 상승)',
        'decision' : '승인/거절',
        'action_guide' : '상담원이 해야할 행동 가이드'
    }}
    '''
    return json.loads(ask_gpt_json(prompt))

decision_results = []
for data in analyzed_data_list:
    decision = make_decision(data, policy_doc)
    # 결과 저장
    final_result = {**data, 'decision_info' : decision}
    decision_results.append(final_result)
    
    print(f"{data['id']} 판단결과")
    print(f"사유 : {decision['reasoning']}")
    print(f"결정 : {decision['decision']}")
    print(f"행동 가이드 : {decision['action_guide']}")
    print('===================================')

1 판단결과
사유 : 1) 해당 요청은 규정 1(단순 변심 환불)과 규정 2(제품 불량 환불) 그리고 규정 3(불만족 보장) 중 어떤 조항이 적용되는지 판단한다. 2) 구매일 2025-10-03에서 현재일 2025-11-24까지 경과일은 52일로, 7일 이내/30일 이내 조건을 모두 초과한다. 3) 박스가 이미 버려졌으므로 1번은 불가하다. 4) 2번 역시 기간 초과로 불가하다. 5) 다판다의 불만족 보장(3) 항목은 구매 확정 후에는 환불이 불가하나 소정의 적립금(2,000원) 지급은 가능하므로 환불 대신 2,000원 적립금을 제안하는 것이 정책상 합리하다. 따라서 이번 요청은 전액 환불은 거절되지만, 2,000원 적립금 제공이 가능한 상황이다.
결정 : 거절
행동 가이드 : 대 상담원 안내: 1) 먼저 정중하게 사과하고, 현재 요청에 대해 정책상 전액 환불은 불가함을 명확히 설명한다. 2) 다판다의 규정 3에 따라 불만족 보상으로 2,000원 적립금을 지급할 수 있음을 안내하고 적립금 지급 절차를 안내한다. 3) 박스 미보유로 인한 1번은 불가함을 재차 언급하되, 1년 이내 무상 AS가 가능하므로 제품의 소음/충전 문제에 대해 AS를 제안한다(AS 가능 여부 확인 필요). 4) 필요 시 주문번호와 고객 정보 확인 후 적립금 처리 및 AS 접수 절차를 진행한다. 5) 고객 피드백(소음/충전 문제)을 시스템에 남겨 향후 품질 개선에 반영하도록 한다.
2 판단결과
사유 : 적용 조항 식별: 1) 단순 변심 환불(구매 후 7일 이내)과 2) 제품 불량 환불(구매 후 30일 이내)은 현재 날짜(2025-11-24) 기준으로 구매일 2025-05-13를 지나 이미 기간 초과. 3) 불만족 보장은 환불이 불가하나 소정의 적립금 2,000원 지급 가능 여부를 포함하고 있음. 다만 이번 요청은 '교환'으로, 회사규정에 교환에 대한 별도의 조항이 명시되어 있지 않으며 AS 규정은 1년 이내 무상 서비스로 별도 교환과 직접 연결되진 않음. 따라서 교환은 규정상 승인 불가이며,

In [9]:
# Prompt Chaining 기법 : 이전 단계의 reasoning을 이용
# 앞의 판단 결과에 따라서 (승인/거절) 톤앤매너를 다르게 해서 이메일을 작성

# 승인: 친절하고 빠르게 안내
# 거절: 단호하게 거절하되 공감하는 톤

def draft_email(final_data):
    # 정보 추출
    product = final_data['product']
    customer_issue = final_data['content']
    decision = final_data['decision_info']['decision']
    reason = final_data['decision_info']['reasoning']
    tone = '매우 정중하고 사과하는 태도' if decision == '승인' else '안타까워하며 공감하지만 원칙을 지키는 태도'

    prompt = f'''
        고객에게 보낼 이메일 작성 해줘

        상황 : 고객이 {product}에 대해 {customer_issue}라고 문의
        결정 : {decision}
        이유 : {reason}
        작성 톤 : {tone}

        이메일 형식 :
        제목 : 문의하신 {product}건에 대한 답변 입니다.
        내용 : (인사 - 공감 - 결정 내용 통보 - 이유를 설명 - 마무리)
    '''
    
    response = client.chat.completions.create(
        model='gpt-5-nano',
        messages=[{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content

for res in decision_results:
    email=draft_email(res)
    print(email)
    print('==========================')

제목: 문의하신 무선 청소기건에 대한 답변 입니다.

안녕하세요, 고객님.

무선 청소기에 대해 시끄럽고 충전이 잘 되지 않는 점으로 불편을 겪으셨다는 말씀에 먼저 깊이 공감드립니다. 고객님의 상황을 이해하고 공정하게 처리하고자 합니다.

다음과 같이 안내드립니다.

결정: 요청하신 전액 환불은 불가합니다.

이유:
- 1) 해당 요청은 규정 1(단순 변심 환불)과 규정 2(제품 불량 환불) 그리고 규정 3(불만족 보장) 중 어떤 조항이 적용되는지 판단이 필요합니다.
- 2) 구매일 2025-10-03에서 현재일 2025-11-24까지의 경과일은 52일로, 7일 이내/30일 이내 조건을 모두 초과합니다.
- 3) 박스가 이미 버려졌으므로 1번은 불가합니다.
- 4) 2번 역시 기간 초과로 불가합니다.
- 5) 다판다의 불만족 보장(3) 항목은 구매 확정 후에는 환불이 불가하나 소정의 적립금(2,000원) 지급은 가능하므로, 환불 대신 2,000원 적립금 제공이 합리적인 처리로 판단됩니다.

따라서 이번 요청은 전액 환불은 거절되지만, 2,000원 적립금을 지급해 드리겠습니다. 이 적립금은 다음 구매 시 사용하실 수 있습니다.

추가로 필요하신 점이나 문의가 있으시면 언제든지 연락 주시길 바랍니다. 감사합니다.

다판다 고객지원팀 올림.
제목: 문의하신 여름용 셔츠건에 대한 답변 입니다.

안녕하세요, 고객님.

사이즈 차이로 인한 불편과 배송 지연으로 걱정을 끼쳐 드려 정말 죄송합니다. 고객님의 문의를 확인하고 아래와 같이 안내 드립니다.

결정 내용
- 이번 건의 교환은 규정상 승인해 드릴 수 없습니다.

이유 설명
- 규정상 교환에 대한 별도 조항이 명시되어 있지 않으며, 단순 변심에 대한 환불은 구매 후 7일 이내, 제품 불량에 대한 환불은 구매 후 30일 이내에 해당합니다. 고객님께서 구매하신 날짜는 2025-05-13로, 현재 날짜(2025-11-24) 기준으로 해당 기간이 이미 경과한 점 양해 부탁드립니다.
- AS 규정은 1년 이내 무상 서비스로 